In [ ]:
## setup code for parsing arguments in my scripts
import argparse
from pathlib import Path
import sys

def main():
    parser = argparse.ArgumentParser(
        description="Extract phonation features from audio files."
    )
    parser.add_argument("input_path", type=Path, help="Path to audio file or folder")
    parser.add_argument("output_file", type=Path, help="Path to output feature file")
    parser.add_argument("static", type=str, help="Static features: 'true' or 'false'")
    parser.add_argument("plots", type=str, help="Plot results: 'true' or 'false'")
    parser.add_argument("format", type=str, choices=["csv", "txt", "npy", "kaldi", "torch"],
                        help="Output file format")

    args = parser.parse_args()

    input_path = args.input_path
    output_file = args.output_file
    static = args.static.lower() == 'true'
    plots = args.plots.lower() == 'true' 
    output_format = args.format

    # Input validation and handling
    if input_path.is_dir() or input_path.is_file():
        print(f"Processing {input_path}")
    else:
        print(f"Input path does not exist {input_path}")
        sys.exit(1)

    print(f"Output stored at {output_file}")
    print(f"Static {static}")
    print(f"Generate plots {plots}")
    print(f"Output format {output_format}")

if __name__ == "__main__":
    main()


In [ ]:
## setup code for a deep learning model

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

# 1. Define a simple feedforward neural network
class MyModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(MyModel, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.net(x)

# 2. Training loop
def train(model, dataloader, criterion, optimizer, device):
    model.train()
    for batch in dataloader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 3. Evaluation loop
def evaluate(model, dataloader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in dataloader:
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            predictions = torch.argmax(outputs, dim=1)
            correct += (predictions == labels).sum().item()
            total += labels.size(0)
    return correct / total

# 4. Main function
def main():
    # Hyperparameters
    input_dim = 128   # e.g., number of features per sample
    hidden_dim = 64
    output_dim = 10   # number of classes
    batch_size = 32
    epochs = 10
    learning_rate = 0.001

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Fake data for demonstration
    X = torch.randn(1000, input_dim)
    y = torch.randint(0, output_dim, (1000,))
    dataset = TensorDataset(X, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # Model, loss, optimizer
    model = MyModel(input_dim, hidden_dim, output_dim).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)

    for epoch in range(epochs):
        train(model, dataloader, criterion, optimizer, device)
        acc = evaluate(model, dataloader, device)
        print(f"Epoch {epoch + 1}: Accuracy = {acc:.2f}")

if __name__ == "__main__":
    main()
